<a href="https://colab.research.google.com/github/Pavitra-khare/DA6401_ASS3_withAtten/blob/main/ASS3_WithAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and login

In [ ]:
!pip install pytorch_lightning
!pip install wandb


In [ ]:
import wandb
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
import torch
import torch.nn.functional as F
torch.cuda.empty_cache()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch.utils.data as data
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as patches
warnings.filterwarnings("ignore")
#function to create 3*3 grid of heatMap
import matplotlib.font_manager as fm


In [ ]:
wandb.login(key="7ff4b917f4a107c44e50219b5fd09ece84a34600")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# Data Preprocessing

In [ ]:
def read_file_0(filepath):
    def process_row(text_row):
        return list(text_row)

    def collect_characters(reader_obj):
        all_chars = []
        for entry in reader_obj:
            all_chars += process_row(entry[0])
        return all_chars

    with open(filepath, mode='r') as file_handle:
        csv_reader = csv.reader(file_handle)
        return collect_characters(csv_reader)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

'''Location of your CSV file (Extracted file)
Location of your CSV file if on kaggle than zip file is fine'''

trainFilepath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_train.csv"
valFilePath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_valid.csv"
testFilePath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_test.csv"

In [ ]:


chars = read_file_0(trainFilepath)
setChar=set(chars)
setChar.add('|')
setOfchar = list(setChar)

# Create the association between characters and their corresponding integer indices
char_to_idx_latin= {char: i+1 for i, char in enumerate(setOfchar)}

In [ ]:
def read_file_1(trainFilepath):
    def extract_chars(text):
        return list(text)

    def accumulate(reader_obj):
        buffer = []
        for entry in reader_obj:
            buffer += extract_chars(entry[1])
        return buffer

    with open(trainFilepath, 'r') as file_stream:
        csv_rows = csv.reader(file_stream)
        return accumulate(csv_rows)

In [ ]:
maxLenDev=0

chars = read_file_1(trainFilepath)
setChar=set(chars)
setChar.add('|')
setOfchar = list(setChar)

charToIndLang ={char: i+1 for i, char in enumerate(setOfchar)}

In [ ]:
with open(trainFilepath, 'r') as csv_file:
    reader = csv.reader(csv_file)
    max_length = 0

    def update_max(current_max, candidate):
        return candidate if candidate > current_max else current_max

    for record in reader:
        length = len(record[0])
        max_length = update_max(max_length, length)

    maxLenEng = max_length

In [ ]:
with open(trainFilepath, 'r') as csv_stream:
    reader_obj = csv.reader(csv_stream)
    longest = 0

    def get_larger(a, b):
        return b if b > a else a

    for line in reader_obj:
        current_length = len(line[1])
        longest = get_larger(longest, current_length)

    maxLenDev = longest

characters in words -> indices












In [ ]:
def convert_characters_to_indices(word, dictionary):
    def safe_lookup(char):
        return dictionary[char] if char in dictionary else -1

    mapped = [safe_lookup(ch) for ch in word]
    filtered = list(filter(lambda idx: idx >= 0, mapped))
    return filtered

In [ ]:
def adjust_sequence_length(indices, maximumLength):
    current_length = len(indices)

    def trim(seq, length):
        return seq[:length]

    def pad(seq, total_length):
        padding_needed = total_length - len(seq)
        return seq + [0] * padding_needed

    if current_length > maximumLength:
        return trim(indices, maximumLength)
    elif current_length < maximumLength:
        return pad(indices, maximumLength)
    return indices


In [ ]:
def convert_indices_to_tensor(indices, dictionary):
    token = dictionary.get('|', 0)

    def add_delimiters(seq, token_id):
        return [token_id] + seq + [token_id]

    sequence = add_delimiters(indices, token)
    tensor_obj = torch.tensor(sequence, device=device)
    return tensor_obj

In [ ]:
def convert_word_to_indices(word, maximumLength, dict):
    char_ids = convert_characters_to_indices(word, dict)

    def standardize_length(seq, target_len):
        return adjust_sequence_length(seq, target_len)

    resized = standardize_length(char_ids, maximumLength)
    tensor_out = convert_indices_to_tensor(resized, dict)
    return tensor_out


In [ ]:
def assign_tensor_to_generated_sequences(sequence):
    tokens = sequence.split()

    def concatenate(tokens):
        return ''.join(tokens)

    def compute_total_length(tokens):
        return sum(len(token) for token in tokens)

    combined = concatenate(tokens)
    total_len = compute_total_length(tokens)

    return combined, total_len

In [ ]:
def assemble_tensor(final_tensor, partition_size=1):
    def safe_partition_size(size):
        return max(1, size)

    chunk_size = safe_partition_size(partition_size)
    segments = [final_tensor[idx:idx + chunk_size] for idx in range(0, len(final_tensor), chunk_size)]

    return segments


In [ ]:
def assemble_assigned_generated_seq(path):
    combined_seq, total_chars = assign_tensor_to_generated_sequences(path)

    def determine_chunk_size(length, divisor=4):
        return max(1, length // divisor)

    segment_length = determine_chunk_size(total_chars)
    partitioned = assemble_tensor(combined_seq, segment_length)

    return partitioned


In [ ]:
def generate_indices(row):
    src_text = row[0]
    tgt_text = row[1]

    def build_indexed_tensor(text, max_len, char_map):
        return convert_word_to_indices(text, max_len, char_map)

    source_tensor = build_indexed_tensor(src_text, maxLenEng, char_to_idx_latin)
    target_tensor = build_indexed_tensor(tgt_text, maxLenDev, charToIndLang)

    return source_tensor, target_tensor
